In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [3]:
cafa3_dir = '/gpfs/alpine/bie108/proj-shared/cafa3/'

In [4]:
from itertools import groupby

def fasta_iter():
    """
    given a fasta file. yield tuples of header, sequence
    """
    with open(os.path.join(cafa3_dir, 'CAFA3_training_data/uniprot_sprot_exp.fasta'), 'r') as fh:
        faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
        for header in faiter:
            headerStr = header.__next__()[1:].strip()
            seq = "".join(s.strip() for s in faiter.__next__())
            yield headerStr, seq
            
seq_df = pd.DataFrame(fasta_iter(), columns=['accession', 'sequence'])
seq_df.shape

(66841, 2)

In [6]:
seq_df.head()

,accession,sequence
0,A0A060X6Z0,MPISSSSSSSTKSMRRAASELERSDSVTSPRFIGRRQSLIEDARKE...
1,A0A068FIK2,MEVGGGSEECCVKVAVHVRPLIGDEKVQGCKDCVTVIPGKPQVQIG...
2,A0A075F932,MVSESHHEALAAPPATTVAAAPPSNVTEPASPGGGGGKEDAFSKLK...
3,A0A078CGE6,MARQMTSSQFHKSKTLDNKYMLGDEIGKGAYGRVYIGLDLENGDFV...
4,A0A086F3E3,MTKGRLEAFSDGVLAIIITIMVLELKVPEGSSWASLQPILPRFLAY...


In [11]:
import os
swissprot_dir = '/gpfs/alpine/bie108/proj-shared/swissprot/'

swissprot = pd.read_parquet(os.path.join(swissprot_dir, 'parsed_swissprot_uniref_clusters.parquet'))
go_terms = pd.read_parquet(os.path.join(swissprot_dir, 'swissprot_quickgo.parquet'))
cafa3_targets = pd.read_csv('cafa3_annotated_targets_with_swissprot.csv.gz', index_col=0)

In [39]:
cafa3_targets[~cafa3_targets.accession.isin(go_terms['GENE PRODUCT ID'])].shape

(188, 7)

In [45]:
len(cafa3_targets)

3328

Of the 3328 CAFA3 targets, 188 are not in the swissprot GO database

In [44]:
seq_df[~seq_df.accession.isin(go_terms['GENE PRODUCT ID'])].shape

(212, 2)

In [47]:
len(seq_df)

66841

Similary, of the 66,841 CAFA3 training targets, 212 are not present in the new GO database

In [56]:
test_accessions = cafa3_targets[cafa3_targets.accession.isin(go_terms['GENE PRODUCT ID'])].accession
train_accessions = seq_df[seq_df.accession.isin(go_terms['GENE PRODUCT ID'])].accession

In [58]:
np.savez_compressed('cafa3_accessions.npz', train=train_accessions.values, test=test_accessions.values)